In [1]:
import time
start_time = time.time()


In [2]:
import sys
import os
sys.path.append(os.path.abspath('../library'))
from multiprocessing import Pool



In [3]:
import sys
sys.path.append('..')  # sobe para a pasta "tactical_analysis"


In [4]:

import pandas as pd 
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient
from library import *
from matplotlib.patches import Polygon as MplPolygon

# Mostrar todas as colunas
pd.set_option('display.max_columns', None)

# Mostrar 10 linhas no head() (o default do pandas já é 5, mas podes mudar)
pd.set_option('display.max_rows', 10)

In [5]:
# ==========================
# 2. Caminho local do ficheiro Parquet
# ==========================
import polars as pl
CAMINHO_PARQUET = "../Dados_Statsbomb/statsbomb_bundesliga_23_24.parquet"  # caminho base

# ==========================
# 3. Ler Parquet (Lazy Polars)
# ==========================
scan = pl.scan_parquet(CAMINHO_PARQUET)  # leitura preguiçosa para performance

# ==========================
# 4. Filtrar match_id específico
# ==========================
match_id_alvo = 3895232  # id alvo
df = (
    scan
    .filter(pl.col("match_id") == match_id_alvo)  # filtro no plano lógico
    .collect()  # materializar em memória só no fim
)

# ==========================
# 5. Verificar resultado
# ==========================
df.select(pl.len().alias("documentos_carregados"))  # mostrar contagem
df.head()  # amostra


actor,id,keeper,location_x,match_id,teammate,visible_area,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_no_touch,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_committed_offensive,foul_committed_penalty,foul_committed_type,foul_won_advantage,foul_won_defensive,…,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_one_on_one,shot_open_goal,shot_outcome,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
bool,str,bool,list[f64],i64,bool,list[f64],struct[1],null,str,bool,bool,bool,bool,bool,list[f64],bool,str,bool,bool,bool,bool,bool,bool,bool,str,bool,str,str,f64,bool,str,bool,bool,str,bool,bool,…,bool,str,i64,str,str,f64,str,i64,str,i64,list[str],i64,bool,str,bool,list[f64],bool,list[struct[4]],str,bool,bool,str,bool,bool,f64,str,str,null,f64,null,f64,null,str,i64,str,str,bool
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[48.33793, 28.361157]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[50.709213, 48.239633]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[52.342012, 33.641217]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[53.934061, 46.318234]",3895232,true,"[120.0, 80.0, … 

In [6]:
from preprocess_utils import converter_visible_area

visible_area_coords = converter_visible_area(df)
visible_area_coords

visible_area
list[list[f64]]
"[[120.0, 80.0], [0.0, 80.0], … [120.0, 80.0]]"
"[[120.0, 80.0], [0.0, 80.0], … [120.0, 80.0]]"
"[[120.0, 80.0], [0.0, 80.0], … [120.0, 80.0]]"
"[[120.0, 80.0], [0.0, 80.0], … [120.0, 80.0]]"
"[[120.0, 80.0], [0.0, 80.0], … [120.0, 80.0]]"
…
"[[65.82355, 80.0], [62.688025, 23.370508], … [65.82355, 80.0]]"
"[[65.82355, 80.0], [62.688025, 23.370508], … [65.82355, 80.0]]"
"[[65.82355, 80.0], [62.688025, 23.370508], … [65.82355, 80.0]]"


In [7]:
# ==========================
# 6. Filtrar por tipo == "Pass" (Polars)
# ==========================
df = df.filter(pl.col("type") == "Pass")  # mantém apenas passes

# ==========================
# 7. Verificar resultado
# ==========================
df.select(pl.len().alias("documentos_carregados"))  # mostra contagem
df.head()  # amostra


actor,id,keeper,location_x,match_id,teammate,visible_area,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_no_touch,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_committed_offensive,foul_committed_penalty,foul_committed_type,foul_won_advantage,foul_won_defensive,…,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_one_on_one,shot_open_goal,shot_outcome,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
bool,str,bool,list[f64],i64,bool,list[f64],struct[1],null,str,bool,bool,bool,bool,bool,list[f64],bool,str,bool,bool,bool,bool,bool,bool,bool,str,bool,str,str,f64,bool,str,bool,bool,str,bool,bool,…,bool,str,i64,str,str,f64,str,i64,str,i64,list[str],i64,bool,str,bool,list[f64],bool,list[struct[4]],str,bool,bool,str,bool,bool,f64,str,str,null,f64,null,f64,null,str,i64,str,str,bool
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[48.33793, 28.361157]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[50.709213, 48.239633]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[52.342012, 33.641217]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,"""Kick Off""",1,"""From Kick Off""","""Leroy Sané""",3053.0,"""Right Attacking Midfield""",2,"""Bayern Munich""",169,"[""13c196fe-41c3-4c6d-b932-ae273d6366ae""]",0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[53.934061, 46.318234]",3895232,true,"[120.0, 80.0, … 

Extrair as coordenadas  X, Y

In [8]:
# ==========================
# 2) Aplicar e validar
# ==========================
df = extrair_coordenadas_passes(df)  # aplicar à tua tabela Polars
df.select(["location_x","pass_end_location","x","y","pass_end_x","pass_end_y"]).head(10)  # ver amostra


location_x,pass_end_location,x,y,pass_end_x,pass_end_y
list[f64],list[f64],f64,f64,f64,f64
"[46.146007, 40.057776]","[27.8, 46.4]",46.146007,40.057776,27.8,46.4
"[48.33793, 28.361157]","[27.8, 46.4]",48.33793,28.361157,27.8,46.4
"[50.709213, 48.239633]","[27.8, 46.4]",50.709213,48.239633,27.8,46.4
"[52.342012, 33.641217]","[27.8, 46.4]",52.342012,33.641217,27.8,46.4
"[53.934061, 46.318234]","[27.8, 46.4]",53.934061,46.318234,27.8,46.4
"[60.314636, 0.948865]","[27.8, 46.4]",60.314636,0.948865,27.8,46.4
"[60.97754, 36.327752]","[27.8, 46.4]",60.97754,36.327752,27.8,46.4
"[61.0, 40.099998]","[27.8, 46.4]",61.0,40.099998,27.8,46.4
"[61.244911, 54.468846]","[27.8, 46.4]",61.244911,54.468846,27.8,46.4


In [9]:
# Selecionar apenas as colunas que queres
df_selected = df[["location_x", "location_y", "pass_end_location", "x", "y", "pass_end_x", "pass_end_y"]]
df_selected

location_x,location_y,pass_end_location,x,y,pass_end_x,pass_end_y
list[f64],list[f64],list[f64],f64,f64,f64,f64
"[46.146007, 40.057776]","[61.0, 40.1]","[27.8, 46.4]",46.146007,40.057776,27.8,46.4
"[48.33793, 28.361157]","[61.0, 40.1]","[27.8, 46.4]",48.33793,28.361157,27.8,46.4
"[50.709213, 48.239633]","[61.0, 40.1]","[27.8, 46.4]",50.709213,48.239633,27.8,46.4
"[52.342012, 33.641217]","[61.0, 40.1]","[27.8, 46.4]",52.342012,33.641217,27.8,46.4
"[53.934061, 46.318234]","[61.0, 40.1]","[27.8, 46.4]",53.934061,46.318234,27.8,46.4
…,…,…,…,…,…,…
"[22.639329, 64.73584]","[21.9, 63.2]","[44.9, 64.4]",22.639329,64.73584,44.9,64.4
"[23.130458, 45.784444]","[21.9, 63.2]","[44.9, 64.4]",23.130458,45.784444,44.9,64.4
"[24.674389, 40.137093]","[21.9, 63.2]","[44.9, 64.4]",24.674389,40.137093,44.9,64.4


#

In [10]:
# determina a orientação da equipa
from detectar_lado import obter_lado_equipa
equipa_esquerda, equipa_direita = obter_lado_equipa(df)


In [11]:
#Corrigir a orientação da equipa
from corrigir_team import corrigir_team_por_frame
df = corrigir_team_por_frame(df)


In [12]:
# ==========================
# 1) Polars: equipa do actor por frame + normalização de coordenadas
# ==========================
import polars as pl

# ==========================
# 1.1) Constantes do campo
# ==========================
FIELD_LENGTH = 120.0  # metros
FIELD_WIDTH  = 80.0   # metros (usar 60.0 se for 120x60)

# ==========================
# 1.2) Derivar equipa do actor por (period, timestamp)
# ==========================
actor_equipa = (
    df
    .filter(pl.col("actor") == True)
    .select(["timestamp", "period", "team"])
    .unique(subset=["timestamp", "period"], keep="first")
    .rename({"team": "equipa_actor_frame"})
)

# ==========================
# 1.3) Juntar essa info ao df
# ==========================
df = df.join(actor_equipa, on=["timestamp", "period"], how="left")

# ==========================
# 1.4) Equipas únicas e equipa do actor no 1.º frame (minuto 0, período 1)
# ==========================
equipas_unicas = (
    df.select(pl.col("team").drop_nulls().unique())
      .to_series(0)
      .to_list()
)

equipa_actor_inicial = (
    df.filter((pl.col("period") == 1) & (pl.col("minute") == 0) & (pl.col("actor") == True))
      .sort("timestamp")
      .select(pl.col("team").first())
      .to_series()
      .item()
)

equipa_oponente = next(e for e in equipas_unicas if e != equipa_actor_inicial)

# ==========================
# 1.5) Assumir orientação: actor inicial à esquerda
# ==========================
equipa_esquerda = equipa_actor_inicial
equipa_direita  = equipa_oponente

# ==========================
# 1.6) Normalizar coordenadas (x_norm, y_norm)
# ==========================
cond_inverte = (
    pl.col("equipa_actor_frame").is_not_null()
    & (pl.col("equipa_actor_frame") != pl.lit(equipa_esquerda))
)

df = df.with_columns([
    pl.when(cond_inverte).then(FIELD_LENGTH - pl.col("x")).otherwise(pl.col("x")).alias("x_norm"),
    pl.when(cond_inverte).then(FIELD_WIDTH  - pl.col("y")).otherwise(pl.col("y")).alias("y_norm"),
])

# ==========================
# 1.7) (Opcional) Seleção rápida para inspeção
# ==========================
# df.select(["period","timestamp","team","equipa_actor_frame","x","y","x_norm","y_norm"]).head()


#Métricas

In [13]:
# ==========================
# Criar coluna 'equipa_jogador' em Polars
# ==========================
import polars as pl

df = df.with_columns([
    pl.when(pl.col("teammate") == True)  # mesmo lado do actor
      .then(pl.col("equipa_actor_frame"))
      .when(pl.col("equipa_actor_frame").is_null())  # sem actor no frame
      .then(None)
      .otherwise(  # lado oposto ao actor
          pl.when(pl.col("equipa_actor_frame") == pl.lit(equipa_esquerda))
            .then(pl.lit(equipa_oponente))
            .otherwise(pl.lit(equipa_esquerda))
      )
      .alias("equipa_jogador")
])

# inspeção rápida
df.select(["period","timestamp","teammate","equipa_actor_frame","equipa_jogador"]).head()


period,timestamp,teammate,equipa_actor_frame,equipa_jogador
i64,str,bool,str,str
1,"""00:00:00.852""",true,"""Bayern Munich""","""Bayern Munich"""
1,"""00:00:00.852""",true,"""Bayern Munich""","""Bayern Munich"""
1,"""00:00:00.852""",true,"""Bayern Munich""","""Bayern Munich"""
1,"""00:00:00.852""",true,"""Bayern Munich""","""Bayern Munich"""
1,"""00:00:00.852""",true,"""Bayern Munich""","""Bayern Munich"""


In [14]:
# ==========================
# Uso 1) Vectorizado (recomendado)
# ==========================
from stretch_index import adicionar_stretch_index

df = adicionar_stretch_index(df)
df.select(["timestamp","period","equipa_jogador","x_norm","y_norm","stretch_index"]).head()


timestamp,period,equipa_jogador,x_norm,y_norm,stretch_index
str,i64,str,f64,f64,f64
"""00:00:00.852""",1,"""Bayern Munich""",46.146007,40.057776,14.37227
"""00:00:00.852""",1,"""Bayern Munich""",48.33793,28.361157,14.37227
"""00:00:00.852""",1,"""Bayern Munich""",50.709213,48.239633,14.37227
"""00:00:00.852""",1,"""Bayern Munich""",52.342012,33.641217,14.37227
"""00:00:00.852""",1,"""Bayern Munich""",53.934061,46.318234,14.37227


#Compactação

In [15]:
# ==========================
# Uso
# ==========================
import polars as pl
from compactacao import adicionar_compactacao

df = adicionar_compactacao(df, equipa_esquerda, equipa_direita)  # compatível com a tua chamada
# df.select(["timestamp","period","equipa_jogador","compactacao"]).head()


In [16]:
# ==========================
# 1. Timestamp alvo
# ==========================
timestamp_escolhido = "00:03:10.690"  # ajusta aqui

# ==========================
# 2. Filtrar e selecionar (Polars)
# ==========================
import polars as pl

cols = ["timestamp", "period", "team", "teammate", "equipa_jogador", "compactacao"]
df_filtrado = (
    df
    .filter(pl.col("timestamp") == pl.lit(timestamp_escolhido))
    .select([c for c in cols if c in df.columns])  # só seleciona as que existem
)

df_filtrado  # mostrar diretamente


timestamp,period,team,teammate,equipa_jogador,compactacao
str,i64,str,bool,str,f64
"""00:03:10.690""",1,"""Bayern Munich""",false,"""Bayer Leverkusen""",1204.32451
"""00:03:10.690""",1,"""Bayern Munich""",false,"""Bayer Leverkusen""",1204.32451
"""00:03:10.690""",1,"""Bayern Munich""",true,"""Bayern Munich""",667.156784
"""00:03:10.690""",1,"""Bayern Munich""",true,"""Bayern Munich""",667.156784
"""00:03:10.690""",1,"""Bayern Munich""",true,"""Bayern Munich""",667.156784
…,…,…,…,…,…
"""00:03:10.690""",1,"""Bayern Munich""",false,"""Bayer Leverkusen""",1204.32451
"""00:03:10.690""",1,"""Bayern Munich""",true,"""Bayern Munich""",667.156784
"""00:03:10.690""",1,"""Bayern Munich""",false,"""Bayer Leverkusen""",1204.32451


#cobertura e equilibrio

In [17]:
# ==========================
# Uso (compatível com o teu snippet)
# ==========================
import importlib, distancia_media as dm

dm = importlib.reload(dm)  # garantir versão atualizada
df = dm.adicionar_distancia_media(df, equipa_esquerda, equipa_direita)

df.select(["timestamp","period","equipa_jogador","distancia_media_entre_jogadores"]).head()


timestamp,period,equipa_jogador,distancia_media_entre_jogadores
str,i64,str,f64
"""00:00:00.852""",1,"""Bayern Munich""",22.659713
"""00:00:00.852""",1,"""Bayern Munich""",22.659713
"""00:00:00.852""",1,"""Bayern Munich""",22.659713
"""00:00:00.852""",1,"""Bayern Munich""",22.659713
"""00:00:00.852""",1,"""Bayern Munich""",22.659713


#indice de equilibrio

In [18]:
# ==========================
# Uso
# ==========================
from indice_equilibrio import adicionar_indice_equilibrio

df = adicionar_indice_equilibrio(df, equipa_esquerda, equipa_direita)
# df.select(["timestamp","period","equipa_jogador","indice_equilibrio_defensivo"]).head()


/Users/rr/Library/Mobile Documents/com~apple~CloudDocs/00.Data_Science/05.Tese/tactical_analysis/library/indice_equilibrio.py:72: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  equil_df = pl.DataFrame(


#Distância coletiva entre equipas

In [19]:
# ==========================
# Uso
# ==========================
from distancia_coletiva import adicionar_distancia_coletiva

df = adicionar_distancia_coletiva(df, equipa_esquerda, equipa_direita)
# df.select(["timestamp","period","equipa_jogador","distancia_coletiva_entre_equipas"]).head()


/Users/rr/Library/Mobile Documents/com~apple~CloudDocs/00.Data_Science/05.Tese/tactical_analysis/library/distancia_coletiva.py:57: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  dist_df = pl.DataFrame(


#Número de passes que quebram linhas defensivas

In [20]:
import pandas as pd
import numpy as np

from quebra_linha import adicionar_passes_quebram_linha

df = adicionar_passes_quebram_linha(df, equipa_esquerda, equipa_direita)


/Users/rr/Library/Mobile Documents/com~apple~CloudDocs/00.Data_Science/05.Tese/tactical_analysis/library/quebra_linha.py:74: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  passes_quebra_df = pl.DataFrame(


#Criação de Espaço

In [21]:
# ==========================
# Uso
# ==========================
from espaco_criado import adicionar_espaco_criado

df = adicionar_espaco_criado(df)  # area_total padrão = 120*80
# df.select(["compactacao","espaco_criado","keeper"]).head()


In [22]:
df.head()

actor,id,keeper,location_x,match_id,teammate,visible_area,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_no_touch,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_committed_offensive,foul_committed_penalty,foul_committed_type,foul_won_advantage,foul_won_defensive,…,shot_freeze_frame,shot_key_pass_id,shot_one_on_one,shot_open_goal,shot_outcome,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure,x,y,pass_end_x,pass_end_y,equipa_actor_frame,team_corrigido,equipa_actor_frame_right,x_norm,y_norm,equipa_jogador,stretch_index,compactacao,distancia_media_entre_jogadores,indice_equilibrio_defensivo,distancia_coletiva_entre_equipas,passe_quebra_linha,espaco_criado
bool,str,bool,list[f64],i64,bool,list[f64],struct[1],null,str,bool,bool,bool,bool,bool,list[f64],bool,str,bool,bool,bool,bool,bool,bool,bool,str,bool,str,str,f64,bool,str,bool,bool,str,bool,bool,…,list[struct[4]],str,bool,bool,str,bool,bool,f64,str,str,null,f64,null,f64,null,str,i64,str,str,bool,f64,f64,f64,f64,str,str,str,f64,f64,str,f64,f64,f64,f64,f64,i64,f64
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null,46.146007,40.057776,27.8,46.4,"""Bayern Munich""","""Bayern Munich""","""Bayern Munich""",46.146007,40.057776,"""Bayern Munich""",14.37227,1048.011185,22.659713,70.0,26.794706,0,8551.988815
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null,46.146007,40.057776,27.8,46.4,"""Bayern Munich""","""Bayern Munich""","""Bayern Munich""",46.146007,40.057776,"""Bayern Munich""",14.37227,1048.011185,22.659713,70.0,26.794706,0,8551.988815
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null,46.146007,40.057776,27.8,46.4,"""Bayern Munich""","""Bayern Munich""","""Bayern Munich""",46.146007,40.057776,"""Bayern Munich""",14.37227,1048.011185,22.659713,70.0,26.794706,0,8551.988815
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0, 80.0, … 80.0]",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2.768998,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Bayern Munich""",169,"""00:00:00.852""","""Pass""",null,46.146007,40.057776,27.8,46.4,"""Bayern Munich""","""Bayern Munich""","""Bayern Munich""",46.146007,40.057776,"""Bayern Munich""",14.37227,1048.011185,22.659713,70.0,26.794706,0,8551.988815
false,"""50c22c33-b812-4a3b-9c03-017530…",false,"[46.146007, 40.057776]",3895232,true,"[120.0,

In [23]:
# ==========================
# 1. Polars → Pandas (one-liner)
# ==========================
df = (df.collect() if isinstance(df, pl.LazyFrame) else df).to_pandas()  # suporta pl.LazyFrame e pl.DataFrame


In [24]:
df

,actor,id,keeper,location_x,match_id,teammate,visible_area,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,block_offensive,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_no_touch,dribble_nutmeg,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_committed_offensive,foul_committed_penalty,foul_committed_type,foul_won_advantage,foul_won_defensive,foul_won_penalty,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_shot_saved_off_target,goalkeeper_shot_saved_to_post,goalkeeper_success_in_play,goalkeeper_success_out,goalkeeper_technique,goalkeeper_type,index,injury_stoppage_in_chain,interception_outcome,location_y,minute,miscontrol_aerial_won,off_camera,out,pass_aerial_won,pass_angle,pass_assisted_shot_id,pass_body_part,pass_cross,pass_cut_back,pass_deflected,pass_end_location,pass_goal_assist,pass_height,pass_inswinging,pass_length,pass_miscommunication,pass_no_touch,pass_outcome,pass_outswinging,pass_recipient,pass_recipient_id,pass_shot_assist,pass_straight,pass_switch,pass_technique,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_one_on_one,shot_open_goal,shot_outcome,shot_saved_off_target,shot_saved_to_post,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure,x,y,pass_end_x,pass_end_y,equipa_actor_frame,team_corrigido,equipa_actor_frame_right,x_norm,y_norm,equipa_jogador,stretch_index,compactacao,distancia_media_entre_jogadores,indice_equilibrio_defensivo,distancia_coletiva_entre_equipas,passe_quebra_linha,espaco_criado
0,False,50c22c33-b812-4a3b-9c03-0175309809ed,False,"[46.146007007757085, 40.05777550092676]",3895232,True,"[120.0, 80.0, 0.0, 80.0, 0.0, 78.8131164733342...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.768998,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5,None,None,"[61.0, 40.1]",0,None,None,None,None,2.954063,None,Right Foot,None,None,None,"[27.8, 46.4]",None,Ground Pass,None,33.792454,None,None,None,None,Manuel Neuer,5570.0,None,None,None,None,None,Kick Off,1,From Kick Off,Leroy Sané,3053.0,Right Attacking Midfield,2,Bayern Munich,169,[13c196fe-41c3-4c6d-b932-ae273d6366ae],0,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,NaN,None,Bayern Munich,169,00:00:00.852,Pass,None,46.146007,40.057776,27.8,46.4,Bayern Munich,Bayern Munich,Bayern Munich,46.146007,40.057776,Bayern Munich,14.372270,1048.011185,22.659713,70.0,26.794706,0,8551.988815
1,False,50c22c33-b812-4a3b-9c03-0175309809ed,False,"[46.146007007757085, 40.05777550092676]",3895232,True,"[120.0, 80.0, 0.0, 80.0, 0.0, 78.8131164733342...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2.768998,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5,None,None,"[61.0, 40.1]",0,None,None,None,None,2.954063,None,Right Foot,None,None,None,"[27.8, 46.4]",None,Ground Pass,None,33.792454,None,None,None,None,Manuel Neuer,5570.0,None,None,None,None,None,Kick Off,1,From Kick Off,Leroy Sané,3053.0,Right Attacking Midfield,2,Bayern Munich,169,[13c196fe-41c3-4c6d-b932-ae273d6366ae],0,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,NaN,None,Bayern Munich,169,00:00:00.852,Pass,None,46.146007,40.057776,27.8,46.4,Baye

fim das métricas


#Visualização

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from mplsoccer import Pitch
from IPython.display import HTML
from matplotlib.lines import Line2D

mpl.rcParams['animation.embed_limit'] = 100_000_000

# 1. Preparar dados

df_minuto = (
    df[df["minute"] == 3]
    .sort_values("timestamp")
    .reset_index(drop=True)
)


df_minuto = (
    df.sort_values("timestamp")
      .reset_index(drop=True)
)


df_minuto["timestamp"] = pd.to_datetime(df_minuto["timestamp"], format="%H:%M:%S.%f", errors="coerce")

# 2. Timestamps únicos
ranked_timestamps = (
    df_minuto[["period", "timestamp"]]
    .drop_duplicates()
    .sort_values(by=["period", "timestamp"])
    .reset_index(drop=True)
)

# 3. Equipas
equipa_azul = df_minuto[df_minuto["actor"] == True]["team"].dropna().unique()[0]
equipas_unicas = df["team"].dropna().unique().tolist()
equipa_vermelha = [e for e in equipas_unicas if e != equipa_azul][0]

# 4. Campo
pitch = Pitch(pitch_type='statsbomb', pitch_color='grass', line_color='white')
fig, ax = plt.subplots(figsize=(16, 7))  # Criar figura manualmente

pitch.draw(ax=ax);  # Desenhar o campo no eixo

# 5. Legenda
legend_elements = [
    Line2D([0], [0], marker='o', color='w', label=f"{equipa_azul} (azul)",
           markerfacecolor='blue', markersize=10, markeredgecolor='black'),
    Line2D([0], [0], marker='o', color='w', label=f"{equipa_vermelha} (vermelha)",
           markerfacecolor='red', markersize=10, markeredgecolor='black'),
    Line2D([], [], color='none', label=f"🔵 Posse: {equipa_azul}")
]
ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=3)

# 6. Objetos animados no campo
azuis = ax.scatter([], [], color='blue', s=100, edgecolors='black')
vermelhos = ax.scatter([], [], color='red', s=100, edgecolors='black')
highlight_actor = ax.scatter([], [], s=200, facecolors='none', edgecolors='white', linewidths=2, zorder=5)
bola = ax.scatter([], [], color='white', s=60, edgecolors='black', zorder=6)
#titulo = ax.text(60, 85, '', ha='center', fontsize=12)

highlight_actor = ax.scatter([], [], s=200, facecolors='none', edgecolors='white', linewidths=2, zorder=5)
bola = ax.scatter([], [], color='white', s=60, edgecolors='black', zorder=6)

# Patch para o contorno da visible_area (convex hull)
actor_visible_area_patch = MplPolygon([[0, 0]], closed=True, edgecolor='white',
                                      facecolor='none', linewidth=2, zorder=4)
actor_visible_area_patch.set_visible(False)
ax.add_patch(actor_visible_area_patch)


titulo = fig.text(0.5, 0.015, '', ha='center', va='bottom', fontsize=12, color='black')


actor_nome = ax.text(0, 0, '', color='white', fontsize=10, ha='center', va='bottom', fontweight='bold', zorder=7)

# 7. Caixas de métricas manualmente fora do campo (ajustado)
metricas_esquerda = fig.text(
    0.04, 0.5, '',  # ← em vez de 0.02, agora 0.04 (mais perto do campo)
    ha='left', va='center', fontsize=12, color='white', family='monospace',  # ← fontsize 12 em vez de 10
    bbox=dict(facecolor='black', alpha=0.6, edgecolor='white', boxstyle='round,pad=0.5')  # ← caixa ligeiramente mais redonda e espaçada
)

metricas_direita = fig.text(
    0.96, 0.5, '',  # ← em vez de 0.92, agora 0.96 (mais perto do campo)
    ha='right', va='center', fontsize=12, color='white', family='monospace',
    bbox=dict(facecolor='black', alpha=0.6, edgecolor='white', boxstyle='round,pad=0.5')
)

# 8. Função para extrair coordenadas
def extract_coords(df):
    return df[["x_norm", "y_norm"]].dropna().values.tolist()

plt.close(fig)

In [26]:
from ids_jogadas import atribuir_ids_jogadas

df = atribuir_ids_jogadas(df)


In [27]:
df[['play_code', 'possession_team', 'period','timestamp']].drop_duplicates()

,play_code,possession_team,period,timestamp
0,Bayern Munich_1_1,Bayern Munich,1,00:00:00.852
200,Bayern Munich_1_1,Bayern Munich,1,00:00:05.246
421,Bayern Munich_1_1,Bayern Munich,1,00:00:08.614
602,Bayern Munich_1_1,Bayern Munich,1,00:00:13.273
802,Bayern Munich_1_1,Bayern Munich,1,00:00:16.267
...,...,...,...,...
153565,Bayern Munich_2_24,Bayern Munich,2,00:48:50.491
153786,Bayern Munich_2_24,Bayern Munich,2,00:48:52.648
154007,Bayern Munich_2_24,Bayern Munich,2,00:49:02.726
154228,Bayern Munich_2_24,Bayern Munich,2,00:49:25.824


In [28]:
#df.to_csv("statsbomb_bundesliga_23_24.csv", index=False)

In [29]:
from voronoi_global import desenhar_voronoi_global


In [30]:
from eps_zones import calcular_zonas_eps_fixed as calcular_eps_polygons


In [31]:
df = atribuir_ids_jogadas(df)


In [32]:
df.x_norm.unique()

array([46.14600701, 48.3379301 , 50.70921302, ..., 95.32561118,
       90.06123989, 85.9771381 ], shape=(16981,))

In [33]:
df[['play_code', 'possession_team', 'period','timestamp']].drop_duplicates()

,play_code,possession_team,period,timestamp
0,Bayern Munich_1_1,Bayern Munich,1,00:00:00.852
200,Bayern Munich_1_1,Bayern Munich,1,00:00:05.246
421,Bayern Munich_1_1,Bayern Munich,1,00:00:08.614
602,Bayern Munich_1_1,Bayern Munich,1,00:00:13.273
802,Bayern Munich_1_1,Bayern Munich,1,00:00:16.267
...,...,...,...,...
153565,Bayern Munich_2_24,Bayern Munich,2,00:48:50.491
153786,Bayern Munich_2_24,Bayern Munich,2,00:48:52.648
154007,Bayern Munich_2_24,Bayern Munich,2,00:49:02.726
154228,Bayern Munich_2_24,Bayern Munich,2,00:49:25.824


In [34]:
'''
from eps_utils import extract_coords
from eps_plotting import desenhar_area_visivel
from update_grid_hex import criar_update
from grid_hex import draw_hex_grid

from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PolyCollection
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML
import pandas as pd
import numpy as np
import math

# --- Limpar patches antigos ---
for patch in ax.patches[:]:
    if isinstance(patch, MplPolygon):
        patch.remove()

# Patch para área visível
visible_poly_patch = MplPolygon([[0, 0]], closed=True, edgecolor='red',
                                facecolor='none', linewidth=1.5, zorder=4)
visible_poly_patch.set_visible(False)
ax.add_patch(visible_poly_patch)

# Desenhar grelha hexagonal
draw_hex_grid(ax, field_width=120, field_height=80)

# Criar função update com área visível e sem zonas EPS
update = criar_update(
    ranked_timestamps, df_minuto,
    azuis, vermelhos, highlight_actor, bola, actor_nome,
    visible_poly_patch,
    titulo, metricas_esquerda, metricas_direita,
    equipa_azul, equipa_vermelha, equipa_esquerda
)

# Criar animação
ani = FuncAnimation(fig, update,
                    frames=len(ranked_timestamps),
                    interval=500, blit=False, repeat=False)

plt.close(fig)
HTML(ani.to_jshtml())
'''

"\nfrom eps_utils import extract_coords\nfrom eps_plotting import desenhar_area_visivel\nfrom update_grid_hex import criar_update\nfrom grid_hex import draw_hex_grid\n\nfrom matplotlib.patches import Polygon as MplPolygon\nfrom matplotlib.collections import PolyCollection\nfrom matplotlib.animation import FuncAnimation\nimport matplotlib.pyplot as plt\nfrom IPython.display import HTML\nimport pandas as pd\nimport numpy as np\nimport math\n\n# --- Limpar patches antigos ---\nfor patch in ax.patches[:]:\n    if isinstance(patch, MplPolygon):\n        patch.remove()\n\n# Patch para área visível\nvisible_poly_patch = MplPolygon([[0, 0]], closed=True, edgecolor='red',\n                                facecolor='none', linewidth=1.5, zorder=4)\nvisible_poly_patch.set_visible(False)\nax.add_patch(visible_poly_patch)\n\n# Desenhar grelha hexagonal\ndraw_hex_grid(ax, field_width=120, field_height=80)\n\n# Criar função update com área visível e sem zonas EPS\nupdate = criar_update(\n    ranked_t

In [35]:
from eps_utils import extract_coords
from eps_plotting import desenhar_area_visivel
from update_grid_hex import criar_update
from grid_hex import draw_hex_grid

from matplotlib.patches import Polygon as MplPolygon
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

# --- Garantir fig/ax ---
try:
    fig
    ax
except NameError:
    fig, ax = plt.subplots(figsize=(10, 7))

# --- Limpar patches antigos ---
for patch in ax.patches[:]:
    if isinstance(patch, MplPolygon):
        patch.remove()

# Patch para área visível (mantido para manter o mesmo pipeline)
visible_poly_patch = MplPolygon([[0, 0]], closed=True, edgecolor='red',
                                facecolor='none', linewidth=1.5, zorder=4)
visible_poly_patch.set_visible(False)
ax.add_patch(visible_poly_patch)

# Desenhar grelha hexagonal (como no original)
draw_hex_grid(ax, field_width=120, field_height=80)

# Criar a função update (processamento/preparação igual ao original)
update = criar_update(
    ranked_timestamps, df_minuto,
    azuis, vermelhos, highlight_actor, bola, actor_nome,
    visible_poly_patch,
    titulo, metricas_esquerda, metricas_direita,
    equipa_azul, equipa_vermelha, equipa_esquerda
)

# --- NÃO correr animação nem chamar update ---
# (removido) ani = FuncAnimation(...)
# (removido) HTML(ani.to_jshtml())

# Opcional: mostrar apenas o estado estático da grelha
plt.show()


Constuir a grelha

In [36]:
from grid_hex import get_hex_centers, _hex_corner
from shapely.geometry import Polygon

# Parâmetros
hex_size = 1.86
field_width = 120
field_height = 80
x0, y0 = 0.0, 0.0

# Obter centros das células
hex_centers = get_hex_centers(field_width, field_height, x0, y0, hex_size)

# Criar polígono de cada célula
cell_polygons = []
for center in hex_centers:
    polygon = Polygon([_hex_corner(center, hex_size, i) for i in range(6)])
    cell_polygons.append(polygon)

# Confirmar visualmente os primeiros resultados
for i in range(3):
    print(f"Célula {i} centro: {hex_centers[i]}, área: {cell_polygons[i].area:.2f}")


Célula 0 centro: (0.0, np.float64(0.0)), área: 8.99
Célula 1 centro: (0.0, np.float64(3.2216145020781117)), área: 8.99
Célula 2 centro: (0.0, np.float64(6.4432290041562235)), área: 8.99


In [37]:
from shapely.geometry import Point, Polygon
import pandas as pd
import numpy as np

# --- 1. Geração da grelha de hexágonos ---

def _hex_corner(center, size, i):
    angle_rad = np.deg2rad(60 * i)
    return (
        center[0] + size * np.cos(angle_rad),
        center[1] + size * np.sin(angle_rad)
    )

def gerar_cell_polygons(field_width=120.0, field_height=80.0, x0=0.0, y0=0.0, hex_size=1.86):
    col_step = 1.5 * hex_size
    row_step = np.sqrt(3) * hex_size
    y_start = (np.sqrt(3) / 2) * hex_size

    num_cols = max(1, int((field_width - hex_size) / col_step) + 1)
    num_rows = max(1, int((field_height - y_start) / row_step) + 1)

    cell_polygons = []
    cell_centers = []

    for col in range(num_cols):
        for row in range(num_rows):
            cx = x0 + col * col_step
            cy = y0 + row * row_step + (col % 2) * (row_step / 2)

            center = (cx, cy)
            corners = [_hex_corner(center, hex_size, i) for i in range(6)]

            cell_polygons.append(Polygon(corners))
            cell_centers.append(center)

    return cell_polygons, cell_centers

# Criar grelha de hexágonos
cell_polygons, cell_centers = gerar_cell_polygons()
n_cells = len(cell_polygons)

# --- 2. Área visível da câmara ---

def to_xy_tuple(c):
    while isinstance(c, (list, tuple, np.ndarray)) and len(c) == 1:
        c = c[0]
    if isinstance(c, (list, tuple, np.ndarray)) and len(c) >= 2:
        return (float(c[0]), float(c[1]))
    raise ValueError(f"Coordenada inválida: {c}")

camera_poly = Polygon([to_xy_tuple(pt) for pt in visible_area_coords[0]])

# --- 3. Função para processar 1 frame ---

def processar_frame(frame_data):
    grid_A = [0] * n_cells
    grid_B = [0] * n_cells
    grid_ball = [False] * n_cells
    grid_visible = [False] * n_cells

    df_azuis = frame_data[frame_data["teammate"] == True].dropna(subset=["x_norm", "y_norm"])
    df_vermelhos = frame_data[frame_data["teammate"] == False].dropna(subset=["x_norm", "y_norm"])
    df_actor = frame_data[frame_data["actor"] == True].dropna(subset=["x_norm", "y_norm"])

    # Jogadores da equipa A
    for _, row in df_azuis.iterrows():
        p = Point(row["x_norm"], row["y_norm"])
        for i, poly in enumerate(cell_polygons):
            if poly.contains(p):
                grid_A[i] += 1
                break

    # Jogadores da equipa B
    for _, row in df_vermelhos.iterrows():
        p = Point(row["x_norm"], row["y_norm"])
        for i, poly in enumerate(cell_polygons):
            if poly.contains(p):
                grid_B[i] += 1
                break

    # Bola
    for _, row in df_actor.iterrows():
        p = Point(row["x_norm"], row["y_norm"])
        for i, poly in enumerate(cell_polygons):
            if poly.contains(p):
                grid_ball[i] = True
                break

    # Visibilidade (centro da célula dentro do polígono da câmara)
    for i, center in enumerate(cell_centers):
        if camera_poly.contains(Point(center)):
            grid_visible[i] = True

    df_resultado = pd.DataFrame({
        "cell_index": range(n_cells),
        "centro_x": [c[0] for c in cell_centers],
        "centro_y": [c[1] for c in cell_centers],
        "jogadores_A": grid_A,
        "jogadores_B": grid_B,
        "tem_bola": grid_ball,
        "visivel": grid_visible
    })

    return df_resultado

# --- 4. Aplicar a todos os frames do jogo ---

frames_resultado = []

# Garante que tens 'df_minuto' com colunas: timestamp, period, x_norm, y_norm, teammate, actor
frames_unicos = df_minuto[["timestamp", "period"]].drop_duplicates()

for _, row in frames_unicos.iterrows():
    ts = row["timestamp"]
    period = row["period"]
    frame_data = df_minuto[(df_minuto["timestamp"] == ts) & (df_minuto["period"] == period)]
    df_resultado_frame = processar_frame(frame_data)
    df_resultado_frame["timestamp"] = ts
    df_resultado_frame["period"] = period
    frames_resultado.append(df_resultado_frame)

# --- 5. Concatenar os resultados ---

df_resultado_final = pd.concat(frames_resultado, ignore_index=True)

# Verificar os dados
df_resultado_final.head()


ValueError: Coordenada inválida: shape: (2,)
Series: '' [f64]
[
	120.0
	80.0
]

In [ ]:
import pandas as pd

# assumir que já tens o df carregado
# df = pd.read_csv("grid.csv")

# garantir que jogadores_A é numérico
df_resultado_final["jogadores_A"] = pd.to_numeric(df_resultado_final["jogadores_A"], errors="coerce").fillna(0).astype(int)

# pivot: linhas = timestamp, colunas = cell_index, valores = jogadores_A
X_A = (
    df_resultado_final.pivot_table(index="timestamp", columns="cell_index", values="jogadores_A", aggfunc="sum")
      .sort_index()
      .fillna(0)
)

# opcional: ordenar colunas numericamente
X_A.columns = X_A.columns.astype(int)
X_A = X_A.reindex(sorted(X_A.columns), axis=1)

# resultado
X_A.head()


In [ ]:
# Salvar os vetores de ocupação por timestamp num CSV
X_A.to_csv("../tactical_analysis/Files/Matrix.csv")


In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"⏱️ O notebook demorou {elapsed_time:.2f} segundos a correr.")